<a href="https://colab.research.google.com/github/hshuai97/Colab20210803/blob/main/CreateYourOwnDataset_graph_pytorch_geometric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

将train, dev, test的文本实现构建成Data对象的图结构，然后批量处理

Refer: [Youtube video](https://www.youtube.com/watch?v=QLIkOtKS4os)

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

In [ ]:
import os.path as osp
import pickle as pkl
import torch

from tqdm import tqdm
import numpy as np
import pandas as pd

from torch_geometric.data import Dataset, Data
from nltk.tokenize import sent_tokenize, word_tokenize

class MyDataset(Dataset):  # build for 5 datasets
    def __init__(self, root, transform=None, pre_transform=None):
      '''
      root: datasets path, is split into raw_dir (raw dataset) and processed_dir (graph data)
      '''
      super(MyDataset, self).__init__(root, transform, pre_transform)

    @property
    def raw_file_names(self):
      '''
      if return file exists, the download is not triggered
      '''
      for NAME in ['20ng', 'r8', 'r52', 'oh', 'mr']:
        return [NAME+'-dev-stemmed.txt', NAME+'-test-stemmed.txt']

    @property
    def processed_file_names(self):
        ''' 
        if these file exists in raw_dir, processing is skipped
        '''
        return ['not_implemented.pt']

    def download(self):
        pass

    def process(self):
        NAME, MODE = 'mr', 'train'
        self.text, self.label = self.ReadDataset(NAME, MODE)
        
        min_c = {'mr':1, '20ng': 10, 'other':5}  # min_count
        min_count = min_c[NAME]

        if MODE =='train':  # if vocab not exist, build vocab on train set
          if not os.path.exists(f'M:/HieGAT/data2graph/{NAME}/{NAME}-vocab.pkl'):
            vocab = self.buildvocab(self.text, min_count=min_count)  # dict
            with open(f'M:/HieGAT/data2graph/{NAME}/{NAME}-vocab.pkl', 'wb') as f:
              pkl.dump(vocab, f, protocol=pkl.HIGHEST_PROTOCOL)  # save vocab as pickle file

        with open(f'M:/HieGAT/data2graph/{NAME}/{NAME}-vocab.pkl', 'rb') as f:
          vocab = pkl.load(f)
        print(len(vocab))
        # todo


        



    def ReadDataset(self, dataset_name, mode):
        NAME = dataset_name
        MODE = mode
        if NAME not in ['20ng', 'r8', 'r52', 'oh', 'mr']:
          raise ValueError('The dataset is not support')

        PATH = 'M:/HieGAT/datasets/'

        texts = []
        labels = []
        with open(os.path.join(PATH, NAME+'-' + MODE + '-stemmed.txt'), 'r') as f:
            data = f.readlines()

            for line in data:
              line = line.strip()
              t = line.split('\t')
              texts.append(t[1])
              labels.append(t[0])


        print(f'Dataset: {NAME}, Total {MODE} X: {len(texts)}, Total {MODE} Y: {len(labels)}')
        print('*'*50)

        return texts,  labels

    def  buildvocab(self, sample, min_count=5):
      '''
      sample: ['wo xihuan ziran yuyan chuli', 'wo ai shengdu xuexi',  'wo xihuan jiqi xuexi']

      '''
      MIN_COUNT = min_count

      freq = {}
      for i in sample:
        for t in word_tokenize(i):
          if t not in freq:
            freq[t] = 0

      for i in sample:
        for t in word_tokenize(i):
          freq[t] +=1

      del_key = []
      for i in freq:
        if freq[i]<MIN_COUNT:
          del_key.append(i)
      
      for i in del_key:
        freq.pop(i)

      vocab_id = {}
      for i, key in enumerate(freq):
        vocab_id[key] = i
      print(f'vocab_id size: {len(vocab_id)}')
      print('*'*50)
      
      return vocab_id

    def w2id(self, input, vocab):
      ids = []
      for w in input:
        if w in ['.', '!', '?']:
          continue
        if w in vocab:
          ids.append(vocab[w])
        else: 
          ids.append(vocab['<unk>'])
      return ids

    def w2token(text, label):
      print(1)

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data

data_mr = MyDataset(root='M:/HieGAT/data2graph/mr')